# Tokenizer

(samples modified from:  https://huggingface.co/docs/transformers/main/en/preprocessing)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [ ]:
# see: https://huggingface.co/docs/transformers/main/en/preprocessing
encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.")
print(encoded_input['input_ids'])
print(encoded_input['token_type_ids'])
print(encoded_input['attention_mask'])

# Vacabulary and Raw Tokens

In [ ]:
from transformers import GPT2Tokenizer # Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
text = "We like Transformers in 2024!"

# Tokenize the text
token_ids = tokenizer.encode(text, add_special_tokens=True) # Output the token IDs
print("Token IDs:", token_ids)

# Convert token IDs back to raw tokens and output them
raw_tokens = [tokenizer.decode([token_id]) for token_id in token_ids]
print("Raw tokens:", raw_tokens)

# Tokens <> Embeddings

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # Load pre-trained model tokenizer
model = BertModel.from_pretrained('bert-base-uncased') # Load pre-trained model
text = "We like Transformers in 2024!" # Text to be tokenized
input_ids = tokenizer.encode(text, add_special_tokens=True) 
print("Token IDs:", input_ids) # Output the token IDs

# Convert token IDs back to raw tokens and output them
raw_tokens = [tokenizer.decode([token_id]) for token_id in input_ids]
print("Raw tokens:", raw_tokens)

# Convert list of IDs to a tensor
input_ids_tensor = torch.tensor([input_ids])

# Pass the input through the model
with torch.no_grad():
    outputs = model(input_ids_tensor)

# Extract the embeddings
embeddings = outputs.last_hidden_state

# Print the embeddings
print("Embeddings: ", embeddings)

### Dimension der Embeddings.
Für jedes der Token (9) gibt es einen Tensor (einen Vektor) mit 768 Parametern

In [ ]:
embeddings.shape

### Darstellung eines Beispiel-Tensors:

In [ ]:
embeddings[0][1]

# Handling multiple Sentences

In [ ]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_inputs = tokenizer(batch_sentences)

# Wir definieren eine kleine Hilfsfunktion, um die Parameter ausgeben zu können:
def outp (inputs):
    for name in ['input_ids', 'token_type_ids', 'attention_mask']:
        if name in inputs:
            print(f"\n{name}")
            for i in range (len (inputs[name])):
                print(inputs[name][i])

outp(encoded_inputs)

# Padding & Truncation

In [ ]:
encoded_inputs = tokenizer(batch_sentences, padding=True, truncation=True)

In [ ]:
outp(encoded_inputs)

# Word based tokenizer

In [ ]:
text = 'But what about second breakfast?'
tokenizer.tokenize(text)

# Sub-Word based tokenizer

In [ ]:
from transformers import XLNetTokenizer

tokenizer = XLNetTokenizer.from_pretrained("xlnet/xlnet-base-cased")
output = tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")
for i in range (len (output)):
    print(output[i], end=' | ')

- Transformers wurde in zwei Sub-Words aufgeteilt
- Satzzeichen habe eigene Tokens

# Your own Tokenizer

## Einen Textcorpus laden
[Credits to huggingface: https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb]

## Ein Dataset von huggingface laden:
Wir laden zunächst das "Wikitext" Dataset von huggingface. Darin enthalten sind Beispieltexte von Wikipedia.

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("wikitext", name="wikitext-103-raw-v1", split="train")

# Zugriffsmöglichkeiten auf das DataSet

In [ ]:
dataset

In [ ]:
dataset[30000]

In [ ]:
dataset[30000:30005]

# DataSet für den Trainnigsprozess bereitstellen (Corpus)
## A) als Liste von Listen
Dies lässt sich einfach umsetzen, hat aber den Nachteil, dass die Daten im Hauptspeicher gehalten werden müssen.

In [ ]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

## B) Als Iterator
Ein (Python-) Iterator ist eine Möglichkeit, Daten sukzessive nach und nach zu laden. Da das DataSet die Daten auf der Festplatte vorhält, wird für den Iterator immer nur ein Teil der Daten in den Hauptspeicher geladen.

Wir definieren diesen hier als Funktion:

In [ ]:
def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

# Verschiedene Tokenizer aufbauen
# 1) Die Architektur eines vorhandenen Tokenizers clonen aber selbst trainieren
[Credits to: https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Wir stellen sicher, dass wir eine "fast" Version der Architektur geladen haben, um das Training im Rahmen zu halten
if not tokenizer.is_fast:
    raise ("Dieser Tokenizer ist nicht geeignet.")
else:
    print("Dieser Tokenizer ist geeignet.")

In [ ]:
gpt_clone_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=25000)

In [ ]:
outp(gpt_clone_tokenizer(dataset[30000:30005]["text"]))

### Diesen (trainierten!) Tokenizer können wir auch zur Wiederverwertung speichern:

In [ ]:
gpt_clone_tokenizer.save_pretrained("../data/gpt_clone_tokenizer")

## 2) Architektur eines Tokenizers selbst aufbauen
### Beispiel: Byte Pair Encodings (wie für GPT-2)

In [ ]:
# Benötigte Imports
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

## Tokenizer-Pipeline
"Um zu verstehen, wie man einen Tokenizer von Grund auf erstellt, müssen wir ein wenig mehr in die Tokenizer-Bibliothek und die Tokenisierungs-Pipeline eintauchen. Diese Pipeline besteht aus mehreren Schritten:

* **Normalisierung**: Führt alle anfänglichen Transformationen über die anfängliche Eingabezeichenkette aus. Wenn Sie z.B. einen Text klein schreiben, ihn vielleicht entfernen oder einen der üblichen Unicode-Normalisierungsprozesse anwenden wollen, fügen Sie einen Normalizer hinzu.
* **Pre-Tokenizer**: Verantwortlich für die Aufteilung der ursprünglichen Eingabezeichenfolge. Das ist die Komponente, die entscheidet, wo und wie die ursprüngliche Zeichenkette vorsegmentiert wird. Das einfachste Beispiel wäre, einfach an Leerzeichen zu trennen.
* **Modell**: Übernimmt die gesamte Erkennung und Generierung von Sub-Token. Dieser Teil kann trainiert werden und ist wirklich von den Eingabedaten abhängig.
* **Post-Processing**: Bietet erweiterte Konstruktionsfunktionen, um mit einigen der Transformers-basierten SoTA-Modelle kompatibel zu sein. Für BERT wird der tokenisierte Satz beispielsweise um [CLS]- und [SEP]-Token "verpackt"."

Vgl. https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb

Und in die umgekehrte Richtung:

**Dekodierung**: Verantwortlich für die Rückführung einer tokenisierten Eingabe in die ursprüngliche Zeichenkette. Der Decoder wird in der Regel nach dem Pre-Tokenizer ausgewählt, den wir zuvor verwendet haben.
Für das Training des Modells bietet die 🤗 Tokenizer-Bibliothek eine Trainer-Klasse, die wir verwenden werden.

Alle diese Bausteine können kombiniert werden, um funktionierende Tokenizer-Pipelines zu erstellen. 

**Beispiele**:
- GPT-2 == BPE-Tokenizer
- BERT == WordPiece-Tokenizer
- T5 == Unigram-Tokenizer


### Ein BPE-Tokenizer benötigt einen Pre-Tokenizer (word based tokens)

In [ ]:
# Basismodell ist ein BPE-Tokenizer
tokenizer = Tokenizer(models.BPE())

In [ ]:
print (tokenizer.pre_tokenizer)
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

In [ ]:
# BPE Byte Level Pre-Tokenizer werden über huggingface zur Verfügung gestellt.
tokenizer.pre_tokenizer.pre_tokenize_str("Tokenizer sind toll!")
# add_prefix_space=False verhindert, dass vor dem ersten Wort ebenfalls das Trennzeichen 'Ġ' erscheint. Alle weiteren Token werden mit diesem Zeichen eingeleitet.
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)


In [ ]:
# Ergebnisse des Pre-Tokenizers anziegen
tokenizer.pre_tokenizer.pre_tokenize_str("Tokenizer sind toll!")

### Unseren Tokenizer trainieren
Der Tokenizer muss trainiert werden, denn er muss die Byte Pairs lernen.

In [ ]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

### Post-Prozessing
Der Tokenizer erhält noch ein Post-Prozessing.
Der ByteLevel PostProcessor kümmert sich um das Trimming der Offsets. Whitespaces können optional ebenfalls getrimmt werden.


In [ ]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

### Tokenizer für Huggingface Framework "verpacken"
HuggingFace stellt Wrapper zur Verfügung, die sicherstellen, dass die Tokenizer zu den APIs der Modelle passen.

Hier verwenden wir einen GPT2TokenizerFast.


In [ ]:
from transformers import GPT2TokenizerFast

custom_gpt_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

### Neuen Tokenizer anwenden

In [ ]:
outp(custom_gpt_tokenizer(dataset[30000:30005]["text"]))